# Dataset Analysis with Item Response Theory

Item Response Theory (IRT) is a statistical framework from psychometrics which models the relationship between test takers and their responses to test questions.
It models the fact that different test items have different levels of difficulty, and differing abilities to discriminate between skilled and unskilled students.

By treating `syftr` evaluation data as a test, and `syftr` flows as 'students', we can gain deeper insight into our flows and evaluation data.

IRT can help us identify:
* Mislabeled evaluation items
* High and low-impact evaluation items
* Overly difficult or overly easy evaluation datasets
* High-skill flows which might not stand out from the accuracy picture alone

This notebook will walk you through applying a simple 2-Parameter Logistic (2pl) IRT model to your study and interpreting its results.

## Your Study Here
Set the name of the study you would like to examine here.

Studies should have 500 or more completed trials for best results.

In [ ]:
STUDY_NAME = "silver1--in-sample--bright_hf--psychology"

In [ ]:
import random
import time

import numpy as np
import pandas as pd
import pyro
import torch
from py_irt.dataset import Dataset as IRTDataset
from py_irt.training import console, IrtModelTrainer, IrtConfig, Path, write_json

from syftr import api

pd.set_option("display.max_rows", 200)


In [ ]:
def get_study_dataset_flows(study_name: str):
    study = api.Study.from_db(STUDY_NAME)
    dataset = study.study_config.dataset
    flows_df = study.flows_df
    return study, dataset, flows_df

def get_item_response_matrix(flows_df):
    item_response_matrix = pd.DataFrame(
        list(flows_df.user_attrs_metric_eval_results)
    ).map(lambda x: x['passing'], na_action='ignore')
    
    return item_response_matrix

In [ ]:
study, dataset, flows_df = get_study_dataset_flows(STUDY_NAME)
# Inspect the flows dataframe
console.print(f"syftr flows dataframe for study: {STUDY_NAME}")
flows_df.T

## Item Response Matrix
The Item Response Matrix is the core data structure of IRT.
The columns represent individual test items, while the rows represent test-takers, and values represent the item evaluation - correct or incorrect.

Note that you may have some `NaN` values in your item response matrix.
This is the case for trials which were halted for cost or performance reasons.

In [ ]:
item_response_matrix = get_item_response_matrix(flows_df)
console.print(f"# Item Response Matrix for study: {STUDY_NAME}")
item_response_matrix

## Measure IRT Parameters

Training an IRT model produces item difficulty and discrimination scores, as well as student (flow) ability scores.

These variables have abstract units, but it's useful to note that the ability and difficulty scores are related.
A student with ability matching the difficulty of a question is estimated to have a 50% chance of correctly answering the question.

This cell will train an IRT model on your data using `py-irt`, `pyro`, and `pytorch`.

Use `device="cpu"` if your computer doesn't have CUDA available.

In [ ]:
def train_irt_model(
    item_response_matrix: pd.DataFrame,
    epochs=2000,
    seed=1234,
    device="cuda", # alternative: "cpu"
    output_dir="./dataset_irt_analysis"
):
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    dataset = IRTDataset.from_pandas(
        item_response_matrix.reset_index(),
        subject_column="index",
        item_columns=item_response_matrix.columns,
    )
    config = IrtConfig(
        model_type="2pl",
        epochs=epochs,
        log_every=100,
        priors=None,
        initializers=None,
        dims=None,
        lr=0.1,
        lr_decay=0.9999,
        dropout=0.5,
        hidden=500,
        vocab_size=None,
        seed=seed,
        deterministic=True,
        model_config={},
    )
    
    trainer = IrtModelTrainer(
        data_path=Path(output_dir),
        config=config,
        dataset=dataset,
        verbose=True,
    )
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    pyro.set_rng_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.use_deterministic_algorithms(True)

    start_time = time.time()
    console.log("Training Model...")
    
    trainer.train(device=device)
    trainer.save(output_dir / "parameters.json")
    write_json(output_dir / "best_parameters.json", trainer.best_params)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    console.log("Train time:", elapsed_time)
    
    return trainer


def predict_item_responses(trainer) -> pd.DataFrame:
    """Predict item response (0 or 1) for subjects in the training dataset."""
    dataset = trainer._dataset

    predictions = trainer.irt_model.predict(
        dataset.observation_subjects,
        dataset.observation_items,
    )

    predictions_df = pd.DataFrame({
        "subject_id": dataset.observation_subjects,
        "example_id": dataset.observation_items,
        "response": dataset.observations,
        "prediction": predictions,
    })
    return predictions_df
    

In [ ]:
trainer = train_irt_model(item_response_matrix, epochs=2000, device="cpu") # device="cuda")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


def print_classifier_metrics(predictions_df, threshold=0.5):
    binary_predictions = (predictions_df['prediction'] >= threshold).astype(int)
    
    accuracy = accuracy_score(predictions_df['response'], binary_predictions)
    precision = precision_score(predictions_df['response'], binary_predictions, zero_division=0)
    recall = recall_score(predictions_df['response'], binary_predictions, zero_division=0)
    f1 = f1_score(predictions_df['response'], binary_predictions, zero_division=0)
    # AUC is already calculated as roc_auc
    
    console.print(f"""--- Classification Metrics (With Prediction Threshold = {threshold}) ---
    Accuracy:               {accuracy:.4f}
    Precision:              {precision:.4f}
    Recall (Sensitivity):   {recall:.4f}
    F1-score:               {f1:.4f}
    """)

    
def plot_prediction_distributions(predictions_df):
    plt.figure()
    sns.kdeplot(predictions_df[predictions_df['response'] == 0]['prediction'], label='Actual: 0 (Incorrect)', fill=True)
    sns.kdeplot(predictions_df[predictions_df['response'] == 1]['prediction'], label='Actual: 1 (Correct)', fill=True)
    plt.title('Distribution of Predicted Probabilities by Actual Class', fontsize=16)
    plt.xlabel('Predicted Probability of Positive Class (1)', fontsize=14)
    plt.ylabel('Density', fontsize=14)
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.7)
    plt.show()


def plot_roc_curve(predictions_df):
    fpr, tpr, thresholds_roc = roc_curve(predictions_df['response'], predictions_df['prediction'])
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Chance')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=14)
    plt.ylabel('True Positive Rate (Sensitivity/Recall)', fontsize=14)
    plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=16)
    plt.legend(loc="lower right")
    plt.grid(True, linestyle=':', alpha=0.7)
    plt.show()


def plot_confusion_matrix(predictions_df, threshold=0.5):
    binary_predictions = (predictions_df['prediction'] >= threshold).astype(int)
    cm = confusion_matrix(predictions_df['response'], binary_predictions)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Predicted 0', 'Predicted 1'],
                yticklabels=['Actual 0', 'Actual 1'])
    plt.ylabel('Actual Class', fontsize=12)
    plt.xlabel('Predicted Class', fontsize=12)
    plt.title(f'Confusion Matrix (Threshold = {threshold})', fontsize=15)
    plt.show()

## Evaluate Model Fit

Next we must evaluate the quality of the IRT model fit.

To do this, we treat the model as a classifier which predicts whether a given student will answer a question correctly or incorrectly.

We don't need a perfect classifier, but it should "behave" well.
Here are some general criteria we are hoping to see:

* Distinct peaks in the distribution of predicted probabilities. The more distinct the better, but expect some overlap. Use this plot to help choose a reasonable prediction threshold.
* AUC of above 0.85 (see ROC plot)
* More true positives and true negatives than either false negatives or positives.

If these criteria cannot be met by adjusting the prediction threshold, you most likely need to run more trials.

In [ ]:
predictions_df = predict_item_responses(trainer)

threshold=0.55
print_classifier_metrics(predictions_df, threshold=threshold)
plot_prediction_distributions(predictions_df)
plot_roc_curve(predictions_df)
plot_confusion_matrix(predictions_df, threshold=threshold)

In [ ]:
def get_labeled_qa_pairs(trainer, syftr_dataset) -> pd.DataFrame:
    params = trainer.best_params
    items_df = pd.DataFrame(
        {
            "Question Difficulty": params["diff"],
            "Question Discrimination": params["disc"],
        },
        index=params["item_ids"].values(),
    )
    qa_pairs = list(syftr_dataset.iter_examples())
    qa_pairs_df = pd.DataFrame([qa.model_dump() for qa in qa_pairs]).set_index("id", drop=True)
    qa_pairs_df = pd.concat([qa_pairs_df, items_df], axis=1)
    return qa_pairs_df


def get_labeled_flows(trainer, flows_df) -> pd.DataFrame:
    params = trainer.best_params
    flows_df["Flow Ability"] = params["ability"]
    flows_df["Flow Accuracy"] = flows_df["values_0"]
    flows_df["Flow Cost or Latency"] = flows_df["values_1"]
    return flows_df

## Label the data

Now we apply the IRT model labels (difficulty, discrimination, and ability) to our source data

In [ ]:
qa_pairs_df = get_labeled_qa_pairs(trainer, dataset)
qa_pairs_df

In [ ]:
flows_df = get_labeled_flows(trainer, flows_df)
flows_df.T

# IRT Analysis

## 2PL IRT Model

This notebook uses a two-parameter logistic (2pl) model of item difficulty, discrimination, and ability.
In this model, the probability of a student correctly answering the question is given by the following formula:

    P_correct(student, question) = 1 / (1 + exp(-a * (b - θ)))

Where

* **a - Discrimination parameter**: How precisely the question discriminates between subjects of different ability levels.
* **b - Difficulty parameter**: How difficult the question is.
* **θ - Ability parameter**: The skill level of the subject.

The IRT model we trained above identified the most likely values for these parameters across all of the flows and evaluation questions in our study.

## Analyzing IRT Parameters

The first set of charts to look at are histograms of the difficulty, discrimination, and ability parameters for our questions and flows.

### Difficulty and Ability

The Difficulty and Ability plots share an x-axis because they have the same 'unit'.
We hope to see the Question Difficulty plot cover the range of flow Ability scores, and we expect these values to generally be within the range of -3 to +3.

If the difficulty is always high relative to ability, this might indicate that the test is too hard for the evaluated flows. 
You may consider running the study longer to find higher-ability flows, or altering your search space to add higher-capability LLMs and more sophisticated RAG strategies.

If the difficulty is always low, this indicates that the test is relatively easy and you could benefit from finding harder questions for the flows to answer in order to better distinguish between high-performing flows.

If the ability plot shows multiple distinct peaks or is very highly skewed, this indicates that IRT might not be a good fit for your dataset.
IRT theory assumes that ability is normally distributed in the population.
If the results are highly non-normal, then the conclusions you can draw from the process are quite limited.
You may consider narrowing this analysis to only flows with a particular `rag_mode` or other setting, running your study for more trials, or trying a different dataset.

### Discrimination

Discrimination measures how well a question separates between students with ability below and above the question's difficulty level (it controls the slope of the 2pl logistic function).

You should expect to see discrimination values between 0.0 and 2.0 or so, with 1.0 being fairly good discrimination and 0.0 being a practically useless question, where answering the question correctly or incorrectly has no impact on the estimated skill level of the subject.

If you have many questions with low discrimination, you may consider omitting them from your evaluation dataset to improve evaluations.

#### Negative Discrimination

If there are any questions with negative discrimination, this indicates either a poor model fit (especially if you have very low values - below -5), or you may have mislabeled questions which are only answered 'correctly' by low-ability flows.

In [ ]:
import matplotlib.gridspec as gridspec

def plot_difficulty_discrimination_ability(qa_pairs_df, flows_df, bins=10):
    fig = plt.figure()#figsize=(10, 8))
    gs = gridspec.GridSpec(2, 2, figure=fig, width_ratios=[1, 1], height_ratios=[1, 1])
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1], sharey=ax1)
    ax3 = fig.add_subplot(gs[1, 0], sharex=ax1)
    
    # Plot Question Difficulty on ax1
    ax1.hist(qa_pairs_df['Question Difficulty'], bins=bins)
    ax1.set_title('Question Difficulty')
    ax1.set_ylabel('Frequency')
    ax1.grid(True)
    plt.setp(ax1.get_xticklabels(), visible=True)

    # Plot Question Discrimination on ax2
    ax2.hist(qa_pairs_df['Question Discrimination'], bins=bins)
    ax2.set_title('Question Discrimination')
    ax2.grid(True)
    plt.setp(ax2.get_yticklabels(), visible=True)

    # Plot Flow Ability on ax3
    ax3.hist(flows_df['Flow Ability'], bins=bins)
    ax3.set_title('Flow Ability')
    ax3.set_xlabel('Ability')
    ax3.set_ylabel('Frequency')
    ax3.grid(True)

    # --- 4. Final Touches ---
    fig.suptitle('2PL Model Parameter Histograms')
    # Use tight_layout to ensure plots and titles don't overlap
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
plot_difficulty_discrimination_ability(qa_pairs_df, flows_df)

## Difficulty vs. Discrimination

This scatter plot helps understand the question set a little more.
We hope to see high-discrimination QA pairs across a variety of difficulty levels.

In [ ]:
def plot_qa_scatter(df, use_qtype=True):
    hue_param = None # Default to no color grouping
    plot_title = 'Question Difficulty vs. Discrimination'
    
    if 'qtype' in df.columns and len(df.qtype.unique()) > 1:
        df["Question Type"] = df.qtype
        hue_param = 'Question Type'
        plot_title += ' by Question Type'

    sns.scatterplot(
        data=df,
        x="Question Discrimination",
        y="Question Difficulty",
        hue=hue_param,
        palette="viridis" if hue_param else None,
        s=80,
        alpha=0.8
    )
    
    plt.title(plot_title, fontsize=16)
    plt.xlabel("Question Discrimination", fontsize=12)
    plt.ylabel("Question Difficulty", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.show()

In [ ]:
plot_qa_scatter(qa_pairs_df, use_qtype=True)

## Ability vs Accuracy

A core insight of IRT is that not all questions are the same.
Accuracy scores therefore aren't able to fully compare the actual 'skill' levels of different flows.

The "Ability" measure from IRT provides a more nuanced take on skill.

In this plot, we hope to see a roughly linear correlation between flow ability and flow accuracy.
If the band is narrow and straight, this indicates that accuracy is a decent metric for this study.
If the band has more width and nonlinearity, this could indicate a poor dataset, or that ability may be a better measure for flows compared to accuracy.

In [ ]:
def plot_flow_ability_accuracy(trainer, flows_df):
    plot_title = 'Flow Ability vs Accuracy'

    
    hue_param = None # Default to no color grouping
    if 'params_rag_mode' in flows_df.columns and len(flows_df.params_rag_mode.unique()) >= 2:
        flows_df["RAG Mode"] = flows_df.params_rag_mode
        hue_param = 'RAG Mode'
        plot_title += ' by RAG Mode'

    params = trainer.best_params
    
    flows_df["Flow Ability"] = params["ability"]
    flows_df["Flow Accuracy"] = flows_df["values_0"]
    
    sns.scatterplot(
        data=flows_df,
        x="Flow Accuracy",
        y="Flow Ability",
        hue=hue_param,
        palette="viridis" if hue_param else None,
        s=80,
        alpha=0.6
    )
    
    plt.title(plot_title, fontsize=16)
    plt.xlabel("Flow Accuracy", fontsize=12)
    plt.ylabel("Flow Ability", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.show()

In [ ]:
plot_flow_ability_accuracy(trainer, flows_df)

## Ability and Accuracy vs Cost or Latency

This final pair of plots compares the standard `syftr` Pareto plot of accuracy versus cost or latency (depending on which you have set as your study's second optimization objective), against the plot of _ability_ versus cost or latency.

What we should expect to see here is that plotting ability gives a more balanced distribution of scores in the y axis, giving more nuance to the picture of flow skill levels.

In [ ]:
def plot_accuracy_ability_vs_cost(flows_df):
    """
    Creates two vertically stacked scatter plots sharing an x-axis.

    - Top Plot: Flow Accuracy vs. Flow Cost or Latency
    - Bottom Plot: Flow Ability vs. Flow Cost or Latency

    Args:
        flows_df (pd.DataFrame): A DataFrame that must contain the columns
                                 'Flow Cost or Latency', 'Flow Accuracy', and 'Flow Ability'.
    """
    # --- 1. Create the figure and subplots ---
    # plt.subplots(2, 1) creates a figure with 2 rows and 1 column of subplots.
    # sharex=True ensures that both plots will share the same x-axis,
    # and zooming/panning will be linked.
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5), sharex=True)

    fig.suptitle('Flow Metrics vs. Cost/Latency', fontsize=16)

    # --- 2. Create the Top Plot (Accuracy vs. Cost) ---
    ax1.scatter(flows_df['Flow Cost or Latency'], flows_df['Flow Accuracy'], alpha=0.6)
    ax1.set_ylabel('Flow Accuracy')
    ax1.set_title('Accuracy vs. Cost')
    ax1.grid(True, linestyle='--', alpha=0.5)
    
    # We don't set the x-label here because it's shared with the bottom plot.

    # --- 3. Create the Bottom Plot (Ability vs. Cost) ---
    ax2.scatter(flows_df['Flow Cost or Latency'], flows_df['Flow Ability'], alpha=0.6, color='C1')
    ax2.set_ylabel('Flow Ability')
    ax2.set_xlabel('Flow Cost or Latency') # The shared x-axis label
    ax2.set_title('Ability vs. Cost')
    ax2.grid(True, linestyle='--', alpha=0.5)
    ax2.set_xscale('log')

    # --- 4. Display the Plot ---
    plt.tight_layout(rect=[0, 0.03, 1, 0.96]) # Adjust layout to make room for suptitle
    plt.show()

plot_accuracy_ability_vs_cost(flows_df)

## Direct Review of QA Pairs

Finally, we can analyze each individual QA pair to understand what makes questions easy or hard, discriminating or nondiscriminating.

Questions are sorted from least to most discriminating - we are most interested in the least discriminative questions, because these are the most likely mislabeled or low-value.

In [ ]:
from rich.console import Console
from rich.rule import Rule
from rich.text import Text

def display_questions_with_stats(questions_df: pd.DataFrame, responses_df: pd.DataFrame, item_response_matrix: pd.DataFrame):
    """
    Displays question data with performance stats derived from a responses DataFrame.
    Uses distinct, high-contrast colors for all stats.
    """
    # --- 1. Preparation ---
    df_sorted = questions_df.sort_values(by="Question Discrimination", ascending=True)

    metrics = {
        "Question Difficulty": (df_sorted["Question Difficulty"].min(), df_sorted["Question Difficulty"].max()),
        "Question Discrimination": (df_sorted["Question Discrimination"].min(), df_sorted["Question Discrimination"].max()),
        #"q_guessability": (df_sorted["q_guessability"].min(), df_sorted["q_guessability"].max()),
    }

    def get_color_style(value: float, metric: str, high_is_good: bool = False) -> str:
        """Calculates a color from green (good) to red (bad)."""
        min_val, max_val = metrics[metric]
        if max_val == min_val: return "rgb(173,216,230)"
        normalized = (value - min_val) / (max_val - min_val)
        if high_is_good: normalized = 1 - normalized
        red = int(255 * normalized)
        green = int(255 * (1 - normalized))
        return f"rgb({red},{green},0)"

    # --- 2. Display Loop ---
    console = Console()
    total_subjects = len(responses_df)

    for index, row in df_sorted.iterrows():
        # --- Divider using the DataFrame index as the ID ---
        console.print(Rule(f"[bold white]ID: {index}", style="dim white"))

        # --- Question & Answer ---
        console.print(f"[bold cyan]❓ Question:[/bold cyan] {row['question']}")
        console.print(f"[bold yellow]💡 Labeled Answer:[/bold yellow] [italic]{str(row['answer'])}[/italic]")

        # --- Calculate new stats from responses_df ---
        if index in item_response_matrix.columns:
            question_col = item_response_matrix[index]
            correct_pct = question_col.mean() * 100
            answered_count = question_col.notna().sum()
            response_rate_pct = (answered_count / total_subjects) * 100 if total_subjects > 0 else 0

            # MODIFICATION: Using "bright_magenta" for Response Rate
            stats_text = Text.assemble(
                ("✅ Correct Responses: ", "bold"), (f"{correct_pct:.1f}%", "bright_green"), ("  |  ", "dim"),
                ("👥 Response Rate: ", "bold"), (f"{response_rate_pct:.1f}%", "bright_magenta"), (f" ({answered_count}/{total_subjects})", "dim")
            )
        else:
            stats_text = Text.assemble(("No response data found for this question.", "red"))

        # --- IRT Model Scores ---
        diff_style = get_color_style(row["Question Difficulty"], "Question Difficulty")
        disc_style = get_color_style(row["Question Discrimination"], "Question Discrimination", high_is_good=True)
        #gues_style = get_color_style(row["q_guessability"], "q_guessability")

        irt_scores_text = Text.assemble(
            ("📊 ", "bold"), ("Difficulty: ", "bold"), (f"{row['Question Difficulty']:.2f}", diff_style), (" | ", "dim"),
            ("Discrimination: ", "bold"), (f"{row['Question Discrimination']:.2f}", disc_style), (" | ", "dim"),
            #("Guessability: ", "bold"), (f"{row['q_guessability']:.2f}", gues_style)
        )

        console.print(irt_scores_text)
        console.print(stats_text)


In [ ]:
display_questions_with_stats(qa_pairs_df, flows_df, item_response_matrix)

In [ ]:
flows_df.T